In [214]:
from flask import Flask, request, render_template, url_for, jsonify
import os
import shutil
from data_utils.utils_function import *
from data_utils.get_planet_scenes import search, download_tif_files
import json
import collections

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
collections.Iterable = collections.abc.Iterable
from deploy_model_instances import *
from data_handler import *


class MarineDebrisApp:
    def __init__(self):
        self.app = Flask(__name__)
        self.MAX_CLOUD_PERCENTAGE = 0.5
        self.SEARCH_FOLDER = "searches/2023-05-14_17-21-22.577842"
        #self.model = ModelDeployment(model_weights_path="best_model_efficientnetb4_4.h5")
        self.results_list = []
        self.app.route("/")(self.home)
        self.app.route("/Find_Debris")(self.Find_Debris)
        self.app.route("/display_on_map")(self.display_on_map)
        self.app.route("/area_to_display", methods=['POST'])(self.area_to_display)
        self.app.route('/upload_geojson', methods=['POST'])(self.upload_geojson)
        self.app.route('/get_images')(self.get_images)
        self.app.route('/predict', methods=['POST'])(self.predict)
        self.app.route('/delete_false_positive', methods=['POST'])(self.delete_false_positive)

    def home(self):
        return render_template("index.html", css_file=url_for('static', filename='mapbox-gl-draw.css'),
        js_file=url_for('static', filename='mapbox-gl-draw.js'))

    def Find_Debris(self):
        searches_folder_names = os.listdir('searches/')
        return render_template("dashboard.html", searches_folder_names=searches_folder_names)

    def get_predictions_folders(self):
        folders = os.listdir('static/')
        predictions_folder_names = []
        for folder in folders:
            if "_predictions" in folder:
                predictions_folder_names.append(folder)
        return predictions_folder_names

    def display_on_map(self):
        return  render_template("display_debris_on_map.html", predictions_folder_names=self  .get_predictions_folders())

    def upload_geojson(self):
        posted_file = str(request.files['geojsonFile'].read(), 'utf-8')
        posted_file = json.loads(posted_file)
        geometry_json, start_date_time_formatted, end_date_time_formatted = \
            extract_geometry_timerange_from_json(posted_file)
        q_res = search(geometry=geometry_json, start_date=start_date_time_formatted
                     , end_date=end_date_time_formatted, cc=self.MAX_CLOUD_PERCENTAGE)
        self.SEARCH_FOLDER = download_tif_files(q_res)
        print("files were downloaded to {}".format(self.SEARCH_FOLDER))
        return self.Find_Debris()

    def loads_json(self, path):
        with open(path, 'r') as file:
            coord = json.load(file)
            return coord

    def get_images(self):
        base_name= os.path.basename(self.SEARCH_FOLDER)
        source_dir = os.path.join("static", f'{base_name}_predictions').replace("\\", "/")
        target_dir = os.path.join("static", f'{base_name}_images').replace("\\", "/")

        os.makedirs(target_dir, exist_ok=True)

        # Copy image files from the source directory to the target directory
        for filename in os.listdir(source_dir):
            source_path = os.path.join(source_dir, filename)
            target_path = os.path.join(target_dir, filename)
            shutil.copy2(source_path, target_path)

        # Get the list of image files in the target directory
        image_files = [f for f in os.listdir(target_dir)
                       if os.path.isfile(os.path.join(target_dir, f))
                       and '.jpg' in f]

        image_urls = [{
                        'image_url': f'{target_dir}/{f}',
                        'image_name': f,
                        'image_coordinates':
                            str(self.loads_json(
                                f'{target_dir}/{f.replace("prediction","").replace(".jpg",".json")}'))
                       }
                         for f in image_files]
        return jsonify(image_urls)

    def predict(self):
        folder = request.form.get('folder')
        self.SEARCH_FOLDER = os.path.join("searches", folder).replace("\\", "/")
        base_name= os.path.basename(self.SEARCH_FOLDER)
        predictions_path = os.path.join("static", f'{base_name}_predictions').replace("\\", "/")
        if not os.path.exists(predictions_path):
            model_input = data_load(self.SEARCH_FOLDER)
            self.results_list = self.model.execute_job(input_data=model_input)
            writing_results(self.results_list, predictions_path)
            self.get_images()
        return self.Find_Debris()

    def get_predictions_points(self, path, points):
        prediction_path = os.path.join("static", path)
        for file in os.listdir(prediction_path):
            if ".json" in file:
                coords = self.loads_json(os.path.join(prediction_path, file))
                points.append(coords[0][0])

    def area_to_display(self):
        selected_checkboxes = request.form.getlist('checkboxes')
        points = []

        for path in selected_checkboxes:
            self.get_predictions_points(path, points)

        return  render_template("display_debris_on_map.html", predictions_folder_names=self.get_predictions_folders(), points=json.dumps(points))

    def delete_false_positive(self):
         data = request.get_json()
         images_folder_prediction_url = data.get('imageUrl')
         images_folder_image_url = images_folder_prediction_url.replace("prediction.jpg",".jpg")
         images_folder_json_url = images_folder_image_url.replace(".jpg",".json")
         print(images_folder_prediction_url)
         print(images_folder_image_url)
         print(images_folder_json_url)
         prediction_folder_prediction_url = images_folder_prediction_url.replace("_images", "_predictions")
         prediction_folder_image_url = images_folder_image_url.replace("_images", "_predictions")
         prediction_folder_json_url = images_folder_json_url.replace("_images", "_predictions")
         print(prediction_folder_prediction_url)
         print(prediction_folder_image_url)
         print(prediction_folder_json_url)

         def remove(path):
             try:
                os.remove(path)
             except FileNotFoundError:
                print(f"File '{path}' does not exist.")

         remove(images_folder_prediction_url)
         remove(images_folder_image_url)
         remove(images_folder_json_url)
         remove(prediction_folder_prediction_url)
         remove(prediction_folder_image_url)
         remove(prediction_folder_json_url)
         return 'Image deleted successfully'

    def run(self):
        self.app.run()

In [215]:
marine_debris_app = MarineDebrisApp()
marine_debris_app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [20/May/2023 16:57:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2023 16:57:53] "GET /static/mapbox-gl-draw.js HTTP/1.1" 304 -
127.0.0.1 - - [20/May/2023 16:57:53] "GET /static/mapbox-gl-draw.css HTTP/1.1" 304 -
127.0.0.1 - - [20/May/2023 16:57:56] "GET /Find_Debris HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2023 16:57:56] "GET /static/loading_animation.gif HTTP/1.1" 304 -
127.0.0.1 - - [20/May/2023 16:57:57] "GET /display_on_map HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2023 16:58:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2023 16:58:02] "GET /static/mapbox-gl-draw.js HTTP/1.1" 304 -
127.0.0.1 - - [20/May/2023 16:58:02] "GET /static/mapbox-gl-draw.css HTTP/1.1" 304 -
